Импорт нужных модулей:

In [ ]:
import requests
from pprint import pprint
import time
from fake_useragent import UserAgent
ua = UserAgent(verify_ssl=False)
session = requests.session()
from pprint import pprint
from html import unescape
from bs4 import BeautifulSoup
import re
import time
import random
from collections import Counter

**NB!** Фикбук банит по IP, если слишком часто заходить просто со своего адреса, поэтому нудно следующее:
-  список совбодных ip;
-  цикл, который проверяет, можно ли подключиться к этому адресу

Также на всякий случай использован `UserAgent`.

Благодаря всему этому можно не использовать `time.sleep`: повысится скорость работы программы, что очень хорошо, когда нужно обойти 50 страниц.

Если программа долго работает, часто выводит ошибки доступа, можно или попробовать перезапустить её, или найти новый список адресов.

In [ ]:
proxies = [
    "122.50.5.148:10000",
    "200.160.119.73:8080",
    "36.27.223.80:32520",
    "72.223.168.67:4145",
    "36.27.223.80:30008",
    "72.212.63.101:4145",
    "200.160.119.73:8080",
    "157.25.84.210:8080",
    "122.50.5.148:10000",
    "167.172.180.40:34291",
    "34.90.194.216:3128",
    "185.206.175.24:80",
    "35.178.190.5:8080",
    "161.35.70.249:3128",
    "182.237.18.6:82",
    "59.124.224.180:3128",
    "122.50.5.148:10000",
    "167.172.180.40:34291",
    "178.210.129.150:1234",
    "185.35.101.17:8081",
    "35.203.144.50:5597",
    "167.172.180.46:37598",
    "167.71.41.173:43667",
    "157.230.103.189:34013",
    "35.223.68.68:3333",
    "213.142.218.226:40816",
    "51.158.172.165:8811",
    "94.189.133.93:8080",
    "167.172.191.249:34327",
    "103.114.53.2:8080",
    "45.234.192.224:8080",
    "167.172.109.12:32805",
    "167.172.180.46:38971",
    "43.227.129.193:83",
    "37.187.96.66:8118",
    "103.83.118.10:55443",
    "103.46.233.190:81",
    "139.180.131.23:3128",
    "77.28.97.34:48458",
    "199.247.16.136:8080",
    "180.211.192.58:8080",
    "88.220.104.178:8080",
    "68.183.221.156:39800",
    "46.36.74.60:8080",
    "138.117.84.161:8080",
    "84.42.4.170:8080",
    "186.24.12.194:8080",
    "129.232.134.107:3128",
    "189.39.123.238:8080",
    "49.156.35.22:8080",
    "34.90.194.216:3128",
    "178.62.108.207:3129",
    "220.150.77.91:6000",
    "45.7.255.74:999",
    "35.223.68.68:3333"]

Собственно, из следующей функции важно получить только `link`, то есть шапку с информацией о тексте. Дальше все данные будем получать именно из неё.

Однако, поскольку мне несколько раз попадалась проверка, не робот ли программа, и ничего не выходило, пришлось добавить обход страницы с капчей.

In [ ]:
def parse_page(page_number, z):
    #сюда вставить ссылку с номером страниц
    url = f'https://ficbook.net/find?fandom_filter=fandom&fandom_group_id=1&fandom_ids%5B0%5D=435&deny_other=1&sizes%5B0%5D=1&sizes%5B1%5D=2&sizes%5B2%5D=3&sizes%5B3%5D=4&pages_min=&pages_max=&ratings%5B0%5D=5&ratings%5B1%5D=6&ratings%5B2%5D=7&ratings%5B3%5D=8&ratings%5B4%5D=9&transl=1&statuses%5B0%5D=1&statuses%5B1%5D=2&statuses%5B2%5D=3&directions%5B0%5D={z}&likes_min=&likes_max=&date_create_min=2021-02-25&date_create_max=2021-03-25&date_update_min=2021-02-25&date_update_max=2021-03-25&title=&sort=3&rnd=1742132714&find=Найти!&p={page_number}#result'   
    headers = {'User-Agent': ua.random}
    for p in proxies:
        try:
            print(p)
            known_proxy_ip = p
            proxy = {'http': known_proxy_ip, 'https': known_proxy_ip}
            req = session.get(url, proxies=proxy, headers={'User-Agent': ua.random})
            page = req.text
            page_soup = BeautifulSoup(page, 'html.parser')
            title = page_soup.find_all('title') #заголовок страницы
            link = page_soup.find_all('section', {'class': 'fanfic-thumb-block'}) #шапка текста
            if 'Проверка' in str(title):
                print('Проверка') #такое слово есть только в заголовке страницы с капчей.
            else:
                break #если всё хорошо, выход из цикла
        except Exception as e:
            print(e)
    return link

Эта функция получает из шапки:
-  заголовок `title`;
-  список персонажей `people`;
-  список тегов `tags`.

Список тегов можно использовать сразу, а с персонажами надо ещё поразбираться: некоторые записаны через косую черту, некоторые вообще непонятно как.

In [ ]:
def parse_text(link):
    tags = []
    people = []
    fic_soup = BeautifulSoup(str(link), 'html.parser')
    title = fic_soup.find('h3').text[18:-14]
    my_people = fic_soup.find_all('a', {'class': 'pairing-link'})
    my_tags = fic_soup.find_all('a', {'class': 'tag'})
    for t in my_tags:
        tags.append(t.text)
    for p in my_people:
        people.append(p.text)    
    return people, tags, title

In [ ]:
already_read = []

In [ ]:
all_tags = []
all_people = []

Функция для записи тегов в файл:

In [ ]:
def write_tags(tags, title, file):
    with open(file, encoding = 'utf-8', mode = 'a') as corpus:
        corpus.write('# ')
        corpus.write(str(title))
        corpus.write('\n')
        for t in tags:
            corpus.write(t)
            corpus.write('\n')

Дальше разбираемся с персонажами.

Это словарь, в котором ключ - какое-нибудь из имён персонажа, значение - имя, которым мы хотим его называть. Это нужно потому, что авторы иногда называют героев не полным именем или только фамилией.

In [ ]:
#Тут надо переделать словарь, но я не разбираюсь в персонажах
p_and_p_dict = {
    'Все': '@',
    'упоминается Майкрофт Холмс': 'Майкрофт Холмс',
    'джон': 'Джон Ватсон',
    'шерлок': 'Шерлок Холмс',
    'femШерлок': 'fem!Холмс',
    'femДжеймс': 'fem!Мориарти',
    'Майкрофт Х': 'Майкрофт Холмс',
    'Шерлок Х': 'Шерлок Холмс',
    'упоминается Шерлок': 'Шерлок Холмс',
    'Шерил': 'fem!Холмс',
    'Шерлoк': 'Шерлок Холмс',
    'Шерли Холмс': 'fem!Холмс',
    'Шерлок Холмс': 'Шерлок Холмс',
    'Шерон Холмс': 'fem!Холмс',
    'Джон В': 'Джон Ватсон',
    'ДВ': 'Джон Ватсон',
    'Шерлок Холмс': 'Шерлок Холмс',
    'Мистер Холмс': 'Шерлок Холмс',
    'Шерлок Холмс ': 'Шерлок Холмс',
    'ШерлокХолмс': 'Шерлок Холмс',
    'ШХ': 'Шерлок Холмс',
    'Холмс': 'Шерлок Холмс',
    'Шерлок ': 'Шерлок Холмс',
    'Шерлок': 'Шерлок Холмс',
    'Шерлок Х.': 'Шерлок Холмс',
    '!Шерлок Холмс': 'Шерлок Холмс',
    'Николь Холмс': 'fem!Холмс',
    'И. Холмс': 'fem!Холмс',
    'Кейт Джил Холмс ': 'fem!Холмс',
    'Эмилия Холмс': 'fem!Холмс',
    'fem!Шерлок': 'fem!Холмс',
    'Виктория Шерлок Холмс': 'fem!Холмс',
    'фэмШерлок': 'fem!Холмс',
    'Шерринфорд Холмс': 'fem!Холмс',
    'fem!Майкрофт Холмс': 'fem!Майкрофт Холмс',
    'fem!Шерлок Холмс': 'fem!Холмс',
    'Джон Хэмиш Ватсон': 'Джон Ватсон',
    'Джон': 'Джон Ватсон',
    'Джон,': 'Джон Ватсон',
    'Джон Ватсон ': 'Джон Ватсон',
    'Джон Уотсон ': 'Джон Ватсон',
    'Джон В.': 'Джон Ватсон',
    'Ватсон': 'Джон Ватсон',
    'Джоан': 'fem!Ватсон',
    'Джоан Уотсон': 'fem!Ватсон',
    'Джун Уотсон': 'fem!Ватсон',
    'Джон Ватсон': 'Джон Ватсон',
    'Джоан Ватсон': 'fem!Ватсон',
    'Эмма Ватсон': 'fem!Ватсон',
    'femДжон': 'fem!Ватсон',
    'fem!Джон': 'fem!Ватсон',
    'Джоан': 'fem!Ватсон',
    'fem!Джон Хэмиш Ватсон': 'fem!Ватсон', 
    'фэмДжон': 'fem!Ватсон',
    'Гарриет': 'Гарриет Ватсон',
    'Гарриет Уотсон': 'Гарриет Ватсон',
    'Майкрофт': 'Майкрофт Холмс',
    'Майкрофт Холмс…': 'Майкрофт Холмс',
    'Майкрофт ': 'Майкрофт Холмс',
    'Майкрофт Х.': 'Майкрофт Холмс',
    'намек на Майкрфот Холмс': 'Майкрофт Холмс',
    'МайкрофтХолмс': 'Майкрофт Холмс',
    'леди Смоллвуд': 'Леди Алисия Смоллвуд',
    'инспектор Лестрейд': 'Грегори Лестрейд',
    'Грегори Лестред': 'Грегори Лестрейд',
    'fem!Грегори Лестрейд': 'fem!Лестрейд',
    'Олвен Лестрейд': 'fem!Лестрейд',
    'Грег': 'Грегори Лестрейд',
    'Грег Лестрейд': 'Грегори Лестрейд',
    'Грегори': 'Грегори Лестрейд',
    'Грегори мельком': 'Грегори Лестрейд',
    'Лестрейд': 'Грегори Лестрейд',
    'Джеймс Мориарти': 'Мориарти',
    'Мориарти (упоминается) ': 'Мориарти',
    'Шерли Мориарти ': 'fem!Мориарти',
    'fem!Джеймс Мориарти': 'fem!Мориарти',
    'фэмДжим Мориарти': 'fem!Мориарти',
    'Джеймс Мориарти ': 'Мориарти',
    'Джейми Мориарти': 'Мориарти',
    'Джеймс Мориарти fem': 'fem!Мориарти',
    'Джейн Мориарти': 'fem!Мориарти',
    'Джим': 'Мориарти',
    'Мориати': 'Мориарти',
    'Джим Мориарти': 'Мориарти',
    'Молли': 'Молли Хупер',
    'Молли Хупер;': 'Молли Хупер',
    'Молли Хупер ': 'Молли Хупер',
    'Молли Купер': 'Молли Хупер',
    'ОЖП': 'ОЖП',
    'НЖП': 'ОЖП',
    'Ирэн': 'Ирэн Адлер',
    'Ирэн А': 'Ирэн Адлер',
    'Ирэн А.': 'Ирэн Адлер',
    'Ирен': 'Ирэн Адлер',
    'ИА': 'Ирэн Адлер',
    'Ирен Адлер': 'Ирэн Адлер',
    'Мэри Морстен': 'Мэри Элизабет Морстен',
    'Мэри ЭлизабетМорстен': 'Мэри Элизабет Морстен',
    'Мэри Элизабет  Морстен': 'Мэри Элизабет Морстен',
    'Эвр Холмс': 'Эврус Холмс',
    'Эвер Холмс': 'Эврус Холмс',
    'Эвр': 'Эврус Холмс',
    'Сара': 'Сара Сойер',
    'Себ': 'Себастьян Моран',
    'Себастьян': 'Себастьян Моран',
    'Моран': 'Себастьян Моран',
    'Себастьянна Моран': 'fem!Себастьян Моран',
    'Джону тоже досталось': 'Джон Ватсон',
    'ОМП': 'ОМП',
    'НМП': 'ОМП',
    'так далее.': '@',
    'остальные': '@',
    'все остальные': '@',
    'Остальные мимокрокодилы': '@',
    'остальные присутствующие в сериале': '@',
    'Пара моих персонажей': '@',
    'жертва': '@',
    '???': '@',
    'т.д.и т.п.': '@',
    'многие другие.': '@',
    'с кое-кем': '@',
    'несколько второстепенных': '@',
    'другие комбинации четверых': '@',
    'пока секрет': '@',
    'а остальные пробегали мимо))': '@',
    'и другие...': '@',
    'Вишневый сад': '@',
    'и другие ': '@',
    'и другие': '@',
    'Ожп': 'ОЖП',
    'упоминаются все герои сериала': '@',
    'герои сериала': '@',
    'остальные.': '@',
    'Кое-кто': '@',
    'undefined': '@',
    'кое-кто ещё': '@',
    'другие.': '@',
    'ожп': 'ОЖП',
    'остальные': '@',
    'остальные по мере появления': '@',
    'Элла (мадам в красном берете)': 'Элла',
    'Розамунд Мэри Ватсон': 'Розамунд Мэри',
    'Розамунд Мэри Эдвин': 'Розамунд Мэри',
    'Мэри Ватсон ': 'Розамунд Мэри',
    'Мэри': 'Розамунд Мэри',
    'Мэри Ватсон': 'Розамунд Мэри',
    'Мэри Элизабет Ватсон ': 'Розамунд Мэри',
    'Скотт.': 'Белла Скотт',
    'Юстина Жавелин (омж)': 'Юстина Жавелин',
    'Упоминается Антея': 'Антея',
    'Том Лейтер — учитель анатомии': 'Том Лейтер',
    'ты': 'читатель',
    'Читатель': 'читатель',
    'Ты': 'читатель',
    'другие': '@',
    'так далее': '@',
    'и т': '@',
    'многие другие': '@'
}

Функция, которая чистит список персонажей (честно, мне очень хотелось посмотреть в глаза авторам, которые используют так много странных символов, не предусмотренных, кажется, функционалом сайта).

In [ ]:
def clean(p):
    clean_people = []
    p = p.replace('\\', '/').replace(' и ', '/').replace('|', '/').replace(',', '/').replace(';', '/').replace('+', '/').replace('.', '/')
    pp = p.split('/')
    for pp_two in pp:
        pp_three = pp_two.split('\\')
        for pp_four in pp_three:  
            pp_five = pp_four.split(' и ')
            for pp_six in pp_five:
                pp_seven = pp_six.split('|')
                for pp_eight in pp_seven:
                    pp_nine = pp_eight.split(',')
                    for pp_ten in pp_nine:
                        pp_eleven = pp_ten.split(';')
                        for pp_tw in pp_eleven:
                            pp_thirteen = pp_ten.split(' + ')
                            for n in pp_thirteen:
                                if n != '':
                                    if ']' in n:
                                        nn = n.split(']')[1]
                                    elif '!' in n and 'fem' not in n:
                                        nn = n.split('!')[1]
                                    else:
                                        nn = n
                                    if nn != '':
                                        if nn[0] == ' ':
                                            nn = nn[1:]
                                    clean_people.append(nn)
    return clean_people

Записываем людей в файл:

In [ ]:
def write_people(people, title, file, dictionary, all_people):
    new_people = []
    with open(file, encoding = 'utf-8', mode = 'a') as corpus:
        corpus.write('# ')
        corpus.write(title)
        corpus.write('\n')
        for person in people:
            p = clean(person)
            for new_p in p:
                new_p = new_p.replace('"', '')
                if new_p.endswith(' '):
                    new_p = new_p[:1]
                if new_p.startswith(' '):
                    new_p = new_p[1:]
                if new_p in dictionary.keys():
                    new_p = dictionary[new_p]
                elif 'ОЖП' in new_p or 'ожп' in new_p or 'НЖП' in new_p or 'нжп' in new_p:
                    new_p = 'ОЖП'
                elif 'ОМП' in new_p or 'омп' in new_p or 'НМП' in new_p or 'нмп' in new_p:
                    new_p = 'ОМП'
                new_people.append(new_p)
        for p_p in set(new_people):
            if p_p != '@' and len(p_p) != 1:
                corpus.write(p_p)
                corpus.write('\n')
                all_people.append(p_p)
        return all_people

Цикл, который проходит по нужному числу страниц:

In [ ]:
titles = []
for x in range(51):
    #for z in range(1):
    print('page', x)
    link = parse_page(x, 6)
    for l in link:
        people, tags, title = parse_text(l)
        if title not in already_read:
            print(title)
            already_read.append(title)
            all_tags.extend(tags)
            num = str(len(already_read))
            write_tags(tags, num, 'mix_tags.txt')
            if len(people) >= 2:
                all_people = write_people(people, num, 'mix_people.txt', p_and_p_dict, all_people)

Ещё на всякий случай можно получить списки всех использованных тегов и персонажей. Я пока не придумала, для чего это надо, но, возможно, получится построить какие-нибудь красивые столбчатые диаграммы или что-нибудь ещё в этом роде.

In [ ]:
print(Counter(all_tags))

In [ ]:
import pprint

In [ ]:
c = Counter(all_people).most_common()

In [ ]:
pprint.pprint(c)